<a href="https://colab.research.google.com/github/AMAYadav/job_code_test/blob/main/cointab_test_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''

Business Scenario
You are a data analyst and your client has a large ecommerce company in India (let’s call it X).
X gets a thousand orders via their website on a daily basis and they have to deliver them as fast
as they can. For delivering the goods ordered by the customers, X has tied up with multiple
courier companies in India as delivery partners who charge them some amount per delivery.
The charges are dependent upon two factors:
● Weight of the product
● Distance between the warehouse (pickup location) and customer’s delivery address
(destination location)
On an average, the delivery charges are Rs. 100 per shipment. So if X ships 1,00,000 orders
per month, they have to pay approximately Rs. 1 crore to the courier companies on a monthly
basis as charges.
As the amount that X has to pay to the courier companies is very high, they want to verify if the
charges levied by their Delivery partners per Order are correct.
Input Data


Left Hand Side (LHS) Data (X’s internal data spread across three reports)
● Website order report- which will list Order IDs and various products (SKUs) part of each
order. Order ID is common identifier between X’s order report and courier company
invoice

● SKU master with gross weight of each product-This should be used to calculate total
weight of each order and during analysis compare against one reported by courier
company in their CSV invoice per Order ID. The courier company calculates weight in
slabs of 0.5 KG multiples, so first you have to figure out the total weight of the shipment
and then figure out applicable weight slabs.
For example:
- If the total weight is 400 gram then weight slab should be 0.5
- If the total weight is 950 gram then weight slab should be 1
- If the total weight is 1 KG then weight slab should be 1
- If the total weight is 2.2 KG then weight slab should be 2.5


● Warehouse pincode to All India pincode mapping -(this should be used to figure out
delivery zone (a/b/c/d/e) and during analysis compare against one reported by courier
company in their CSV invoice per Order ID


RHS Data (courier company invoice in CSV file)
● Invoice in CSV file mentioning AWB Number (courier company’s own internal ID), Order
ID (company X’s order ID), weight of shipment, warehouse pickup pincode, customer
delivery pincode, zone of delivery, charges per shipment, type of shipment

● Courier charges rate card at weight slab and pincode level. If the invoice mentions
“Forward charges” then only forward charges (“fwd”) should be applicable as per zone
and fixed & additional weights based on weight slabs. If the invoice mentions “Forward
and rto charges” then forward charges (“fwd”) and RTO charges (“rto”) should be
applicable as per zone and fixed & additional weights based on weight slabs.

----------
● For the first 0.5 KG, “fixed” rate as per the slab is applicable. For each additional 0.5 KG,
“additional” weight in the same proportion is applicable. Total charges will be “fixed” +
“total additional” if any
-----------

#OUTPUT FILES TO BE IN_ORDER



Output Data 1
Create a resultant CSV/Excel file with the following columns:
● Order ID
● AWB Number
● Total weight as per X (KG)
● Weight slab as per X (KG)
● Total weight as per Courier Company (KG)
● Weight slab charged by Courier Company (KG)
● Delivery Zone as per X
● Delivery Zone charged by Courier Company
● Expected Charge as per X (Rs.)
● Charges Billed by Courier Company (Rs.)
● Difference Between Expected Charges and Billed Charges (Rs.)

'''

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [3]:
# from google.colab import drive
# drive.mount('/content/drive/')

# %cd /content/drive/My Drive/cointab/

In [4]:
# x_courier company files

order_report=pd.read_excel('/content/Company X - Order Report.xlsx')
order_report

pincode=pd.read_excel('/content/Company X - Pincode Zones.xlsx')

sku=pd.read_excel('/content/Company X - SKU Master.xlsx')

# courier delievery files

invoice=pd.read_excel('/content/Courier Company - Invoice.xlsx')

rates = pd.read_excel('/content/Courier Company - Rates.xlsx')

In [5]:
# ------------------------------#
b=order_report.merge(sku,on='SKU')[['ExternOrderNo','Order Qty','Weight (g)']]
b['total_weight(kg)']=round((b['Order Qty']*b['Weight (g)'])/1000,2)
b1=b.groupby('ExternOrderNo',as_index=False).sum()
b2=b1.merge(invoice,left_on='ExternOrderNo',right_on='Order ID',how='left')
b2

,ExternOrderNo,Order Qty,Weight (g),total_weight(kg),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001806210,2.0,220,0.22,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5
1,2001806226,2.0,240,0.48,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2
2,2001806229,3.0,500,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2
3,2001806232,8.0,377,1.30,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
4,2001806233,2.0,245,0.25,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3
...,...,...,...,...,...,...,...,...,...,...,...,...
119,2001821995,2.0,477,0.48,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4
120,2001822466,10.0,352,1.38,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0
121,2001823564,6.0,336,0.66,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8
122,2001825261,13.0,611,1.56,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0


In [6]:
b3=b2[['total_weight(kg)',
       'AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)']]

In [73]:
#----------------------------------------#
##
combined=pd.merge(b3,pincode, how='left', left_on=['Warehouse Pincode','Customer Pincode',],right_on = ['Warehouse Pincode','Customer Pincode',])
combined.info();
combined

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_weight(kg)      174 non-null    float64
 1   AWB Code              174 non-null    int64  
 2   Order ID              174 non-null    int64  
 3   Charged Weight        174 non-null    float64
 4   Warehouse Pincode     174 non-null    int64  
 5   Customer Pincode      174 non-null    int64  
 6   Zone_x                174 non-null    object 
 7   Type of Shipment      174 non-null    object 
 8   Billing Amount (Rs.)  174 non-null    float64
 9   Zone_y                174 non-null    object 
dtypes: float64(3), int64(4), object(3)
memory usage: 15.0+ KB


,total_weight(kg),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Zone_y
0,0.22,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,b
1,0.48,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,d
2,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,d
3,1.30,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
4,0.25,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,b
...,...,...,...,...,...,...,...,...,...,...
169,0.48,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4,b
170,1.38,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0,b
171,0.66,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8,d
172,1.56,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0,d


In [9]:
rates

,fwd_a_fixed,fwd_a_additional,fwd_b_fixed,fwd_b_additional,fwd_c_fixed,fwd_c_additional,fwd_d_fixed,fwd_d_additional,fwd_e_fixed,fwd_e_additional,rto_a_fixed,rto_a_additional,rto_b_fixed,rto_b_additional,rto_c_fixed,rto_c_additional,rto_d_fixed,rto_d_additional,rto_e_fixed,rto_e_additional
0,29.5,23.6,33,28.3,40.1,38.9,45.4,44.8,56.6,55.5,13.6,23.6,20.5,28.3,31.9,38.9,41.3,44.8,50.7,55.5


In [11]:
combined

,total_weight(kg),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Zone_y
0,0.22,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,b
1,0.48,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,d
2,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,d
3,1.30,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d
4,0.25,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,b
...,...,...,...,...,...,...,...,...,...,...
169,0.48,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4,b
170,1.38,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0,b
171,0.66,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8,d
172,1.56,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0,d


In [13]:
#---------------------#
# combined['charges']=combined[['Type of Shipment','Zone_x','total_weight(kg)','Charged Weight','Billing Amount (Rs.)']][['Type of Shipment']].replace(
#     ['Forward charges','Forward and RTO charges'],['fwd','fwd_rto'])

#  combined['charges_zones']= combined['charges']+'_'+ combined['Zone_x']

#  combined['total_weight(kg)']

In [ ]:
combined['total_weight(kg)'].unique()

In [ ]:
conditions=[
    (combined['total_weight(kg)']<= 0.5 ),
    (combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 ),
    (combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 ),
    (combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 ),
    (combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 ),
    (combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0 ),
]

choices = ['0.5','1.0','1.5','2.0','2.5','3.0']
combined['weight_slab_x'] = np.select(conditions,choices)

# ---------

conditions=[
    (combined['Charged Weight']<= 0.5 ),
    (combined['Charged Weight']> 0.5) & (combined['Charged Weight']<= 1.0 ),
    (combined['Charged Weight']> 1.0) & (combined['Charged Weight']<= 1.5 ),
    (combined['Charged Weight']> 1.5) & (combined['Charged Weight']<= 2.0 ),
    (combined['Charged Weight']> 2.0) & (combined['Charged Weight']<= 2.5 ),
    (combined['Charged Weight']> 2.5) & (combined['Charged Weight']<= 3.0 ),
]

choices = ['0.5','1.0','1.5','2.0','2.5','3.0']
combined['weight_slab_courier'] = np.select(conditions,choices)

# ---------

conditions=[
    (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='a') & (combined['weight_slab_x'] =='0.5'),
    (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='b') &(combined['weight_slab_x'] == '0.5'),
    (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='c') &(combined['weight_slab_x'] == '0.5'),
    (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='d') &(combined['weight_slab_x'] == '0.5'),
    (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='e') &(combined['weight_slab_x'] == '0.5'),
# ---------
    (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='a') & (combined['weight_slab_x'] =='0.5'),
    (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='b') &(combined['weight_slab_x'] == '0.5'),
    (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='c') &(combined['weight_slab_x'] == '0.5'),
    (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='d') &(combined['weight_slab_x'] == '0.5'),
    (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='e') &(combined['weight_slab_x'] == '0.5'),

# ------------ Fixed charges
 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),
# ---------------------------------b
 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),

# ---------------------------------c
 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),
# --------------------------------d

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),
 # ---------------------------------e
 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),

# -----------------------

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='a') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),

# ---------------------------------b
 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='b') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),

--
# ---------------------------------c
 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='c') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),

# --------------------------------d

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='d') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),


 # ---------------------------------e
 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 0.5) & (combined['total_weight(kg)']<= 1.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 1.0) & (combined['total_weight(kg)']<= 1.5 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 1.5) & (combined['total_weight(kg)']<= 2.0 )),

 (combined['Type of Shipment']== 'Forward and RTO charges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 2.0) & (combined['total_weight(kg)']<= 2.5 )),

 (combined['Type of Shipment']== 'Forward and RTO chargescharges') & (combined['Zone_x'] =='e') & 
 ((combined['total_weight(kg)']> 2.5) & (combined['total_weight(kg)']<= 3.0)),



]

choices = ['29.5','33','40.1','45.4','56.6',
           # --------- fwd + rto
          29.5+13.6, 33+20.5, 40.1+31.9	,45.4+41.3,56.6+50.7,
          #  Forward 
           # ---------a
           29.5+23.6 , 29.5+2*23.6 ,29.5+3*23.6 ,29.5+4*23.6 , 29.5+5*23.6,
          #  -------------b
           33+28.3,33+2*28.3,33+3*28.3,33+4*28.3,33+5*28.3,
          #  ----------c
           40.1+38.9,40.1+2*38.9,40.1+3*38.9,40.1+4*38.9,40.1+5*38.9,
           
          #  ---------------d	
           45.4+44.8,45.4+2*44.8,45.4+3*44.8,45.4+4*44.8,45.4+5*44.8,
          #  -------------e
           56.6+55.5,56.6+2*55.5,56.6+3*55.5,56.6+4*55.5,56.6+5*55.5,

          #  Forward and RTO charges
          #  a
           (29.5+13.6)+(23.6+23.6),(29.5+2*13.6)+(23.6+2*23.6),(29.5+3*13.6)+(23.6+3*23.6),(29.5+4*13.6)+(23.6+4*23.6),(29.5+5*13.6)+(23.6+5*23.6),
          #  b
            (33+28.3)+(20.5+28.3),(33+2*28.3)+ (20.5+2*28.3),(33+3*28.3)+ (20.5+3*28.3),(33+4*28.3)+ (20.5+4*28.3),(33+5*28.3)+ (20.5+5*28.3),
          
          #  c
           (40.1+38.9)+ (31.9+38.9),(40.1+2*38.9)+ (31.9+2*38.9),(40.1+3*38.9)+ (31.9+3*38.9),(40.1+4*38.9)+ (31.9+4*38.9),(40.1+5*38.9)+ (31.9+5*38.9),	
           
          #  d
           (45.4+44.8)+(41.3+44.8),(45.4+2*44.8)+(41.3+2*44.8),(45.4+3*44.8)+(41.3+3*44.8),(45.4+4*44.8)+(41.3+4*44.8),(45.4+5*44.8)+(41.3+5*44.8)
           ,
          #  e 
            (56.6+55.5)+(50.7+55.5),(56.6+2*55.5)+(50.7+2*55.5),(56.6+3*55.5)+(50.7+3*55.5),(56.6+4*55.5)+(50.7+4*55.5),(56.6+5*55.5)+(50.7+5*55.5)
      ]
combined['expected_charges_x(Rs.)'] = np.select(conditions,choices)

# -------------
combined.head(50)

In [ ]:
rates

In [ ]:
combined.columns
combined['Expected_charges_x(Rs.)']=combined['expected_charges_x(Rs.)'].apply(lambda x: float(x))
combined.info()

In [27]:
combined['Difference Between Expected Charges and Billed Charges (Rs.)']=combined['Billing Amount (Rs.)']- combined['expected_charges_x(Rs.)']
combined

,total_weight(kg),AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone_x,Type of Shipment,Billing Amount (Rs.),Zone_y,weight_slab_x,weight_slab_courier,expected_charges_x(Rs.),Difference Between Expected Charges and Billed Charges (Rs.),Expected_charges_x(Rs.)
0,0.22,1091117221940,2001806210,2.92,121003,140604,b,Forward charges,174.5,b,0.5,3.0,33.0,141.5,33.0
1,0.48,1091117222065,2001806226,0.68,121003,723146,d,Forward charges,90.2,d,0.5,1.0,45.4,44.8,45.4
2,0.50,1091117222080,2001806229,0.71,121003,421204,d,Forward charges,90.2,d,0.5,1.0,45.4,44.8,45.4
3,1.30,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,d,1.5,1.5,135.0,0.0,135.0
4,0.25,1091117222135,2001806233,0.78,121003,263139,b,Forward charges,61.3,b,0.5,1.0,33.0,28.3,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,0.48,1091121183730,2001821995,0.50,121003,342008,d,Forward charges,45.4,b,0.5,0.5,45.4,0.0,45.4
170,1.38,1091121305541,2001822466,1.10,121003,342301,d,Forward charges,135.0,b,1.5,1.5,135.0,0.0,135.0
171,0.66,1091121666133,2001823564,0.70,121003,492001,d,Forward and RTO charges,172.8,d,1.0,1.0,176.3,-3.5,176.3
172,1.56,1091121981575,2001825261,1.60,121003,517128,d,Forward and RTO charges,345.0,d,2.0,2.0,355.5,-10.5,355.5


In [28]:
combined.columns

Index(['total_weight(kg)', 'AWB Code', 'Order ID', 'Charged Weight',
       'Warehouse Pincode', 'Customer Pincode', 'Zone_x', 'Type of Shipment',
       'Billing Amount (Rs.)', 'Zone_y', 'weight_slab_x',
       'weight_slab_courier', 'expected_charges_x(Rs.)',
       'Difference Between Expected Charges and Billed Charges (Rs.)',
       'Expected_charges_x(Rs.)'],
      dtype='object')

In [45]:
final=combined[['Order ID','AWB Code','total_weight(kg)','weight_slab_x','Charged Weight','weight_slab_courier','Zone_x','Zone_y','expected_charges_x(Rs.)','Billing Amount (Rs.)','Difference Between Expected Charges and Billed Charges (Rs.)']]
final.rename(
    columns={'total_weight(kg)':'Total_weight_as_per_x_(kg)','weight_slab_x':'weight_slab_as_per_x(kg)','Charged Weight':'Total_weight_as_per_courier_company_(kg)','weight_slab_courier':'weight_slab_charged_by_courier_company','Zone_x':'Delievery_Zone_as_per_x'},
             )

final           


,Order ID,AWB Code,Total_weight_as_per_x_(kg),weight_slab_as_per_x(kg),Total_weight_as_per_courier_company_(kg),weight_slab_charged_by_courier_company,Delievery_Zone_as_per_x,Zone_y,expected_charges_x(Rs.),Billing Amount (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806210,1091117221940,0.22,0.5,2.92,3.0,b,b,33.0,174.5,141.5
1,2001806226,1091117222065,0.48,0.5,0.68,1.0,d,d,45.4,90.2,44.8
2,2001806229,1091117222080,0.50,0.5,0.71,1.0,d,d,45.4,90.2,44.8
3,2001806232,1091117222124,1.30,1.5,1.30,1.5,d,d,135.0,135.0,0.0
4,2001806233,1091117222135,0.25,0.5,0.78,1.0,b,b,33.0,61.3,28.3
...,...,...,...,...,...,...,...,...,...,...,...
169,2001821995,1091121183730,0.48,0.5,0.50,0.5,d,b,45.4,45.4,0.0
170,2001822466,1091121305541,1.38,1.5,1.10,1.5,d,b,135.0,135.0,0.0
171,2001823564,1091121666133,0.66,1.0,0.70,1.0,d,d,176.3,172.8,-3.5
172,2001825261,1091121981575,1.56,2.0,1.60,2.0,d,d,355.5,345.0,-10.5


In [180]:
new=pd.DataFrame(index=['Total orders where X has been correctly charged','Total Orders where X has been overcharged','Total Orders where X has been undercharged']
  ,columns=['Count','Amount(Rs.)'])


new.loc['Total orders where X has been correctly charged','Count'] = final.where(final['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0)[['Difference Between Expected Charges and Billed Charges (Rs.)']].count().values 
new.loc['Total orders where X has been correctly charged','Amount(Rs.)'] = final.where(final['Difference Between Expected Charges and Billed Charges (Rs.)'] == 0)[['Difference Between Expected Charges and Billed Charges (Rs.)']].sum().values 

new.loc['Total Orders where X has been overcharged','Count'] = final.where(final['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0)[['Difference Between Expected Charges and Billed Charges (Rs.)']].count().values
new.loc['Total Orders where X has been overcharged','Amount(Rs.)'] = final.where(final['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0)[['Difference Between Expected Charges and Billed Charges (Rs.)']].sum().values

new.loc['Total Orders where X has been undercharged','Count'] = (combined[['Order ID']].count() - invoice[['Order ID']].count()).values

new.loc['Total Orders where X has been undercharged','Amount(Rs.)'] = (combined[['Billing Amount (Rs.)']].sum() - invoice[['Billing Amount (Rs.)']].sum()).values 

new

,Count,Amount(Rs.)
Total orders where X has been correctly charged,[46],[0.0]
Total Orders where X has been overcharged,[22],[-480.4999999999998]
Total Orders where X has been undercharged,[50],[4784.600000000002]


In [187]:

with pd.ExcelWriter('output.xlsx') as writer:  
    new.to_excel(writer, sheet_name='Sheet_name_2')
    final.to_excel(writer, sheet_name='Sheet_name_1')